In [39]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [40]:
senior_center = pd.read_csv(
    filepath_or_buffer = "C:/Users/jihoo/Desktop/gyeongnam-competition/data/경로당 데이터.csv", 
    encoding = "cp949"
    )
senior_center.head

<bound method NDFrame.head of         연번  시군명             시설명                    주소
0        1  창원시          가곡 경로당  의창구 동읍 신촌본포로442번길 26
1        2  창원시         고양 경로당        의창구 동읍 백월로 540 
2        3  창원시         곡목 경로당   의창구 동읍 동읍로 359번길 43 
3        4  창원시         금동 경로당    의창구 동읍 동읍로 835번길 7 
4        5  창원시         금산 경로당    의창구 동읍 동읍로 670번길 6 
...    ...  ...             ...                   ...
7529  7530  합천군       (용주)성산경로당          용주면 성산3길 31 
7530  7531  합천군     (용주)성산2구경로당           용주면 성산길 18 
7531  7532  합천군       (용주)손목경로당          용주면 손목2길 13 
7532  7533  합천군  (용주)손목1구할머니경로당       용주면 황계폭포로 1395 
7533  7534  합천군     (용주)손목2구경로당          용주면 손목3길 74 

[7534 rows x 4 columns]>

In [41]:
senior_center.columns

Index(['연번', '시군명', '시설명', '주소'], dtype='object')

In [42]:
senior_center["전체 주소"] = "경상남도" + " " + senior_center["시군명"] + " " + senior_center["주소"]
senior_center["전체 주소"].head

<bound method NDFrame.head of 0       경상남도 창원시 의창구 동읍 신촌본포로442번길 26
1            경상남도 창원시 의창구 동읍 백월로 540 
2       경상남도 창원시 의창구 동읍 동읍로 359번길 43 
3        경상남도 창원시 의창구 동읍 동읍로 835번길 7 
4        경상남도 창원시 의창구 동읍 동읍로 670번길 6 
                    ...              
7529            경상남도 합천군 용주면 성산3길 31 
7530             경상남도 합천군 용주면 성산길 18 
7531            경상남도 합천군 용주면 손목2길 13 
7532         경상남도 합천군 용주면 황계폭포로 1395 
7533            경상남도 합천군 용주면 손목3길 74 
Name: 전체 주소, Length: 7534, dtype: object>

In [44]:
senior_center = senior_center.loc[:,['전체 주소']]
senior_center.head

<bound method NDFrame.head of                               전체 주소
0     경상남도 창원시 의창구 동읍 신촌본포로442번길 26
1          경상남도 창원시 의창구 동읍 백월로 540 
2     경상남도 창원시 의창구 동읍 동읍로 359번길 43 
3      경상남도 창원시 의창구 동읍 동읍로 835번길 7 
4      경상남도 창원시 의창구 동읍 동읍로 670번길 6 
...                             ...
7529          경상남도 합천군 용주면 성산3길 31 
7530           경상남도 합천군 용주면 성산길 18 
7531          경상남도 합천군 용주면 손목2길 13 
7532       경상남도 합천군 용주면 황계폭포로 1395 
7533          경상남도 합천군 용주면 손목3길 74 

[7534 rows x 1 columns]>

In [45]:
senior_center.head

<bound method NDFrame.head of                               전체 주소
0     경상남도 창원시 의창구 동읍 신촌본포로442번길 26
1          경상남도 창원시 의창구 동읍 백월로 540 
2     경상남도 창원시 의창구 동읍 동읍로 359번길 43 
3      경상남도 창원시 의창구 동읍 동읍로 835번길 7 
4      경상남도 창원시 의창구 동읍 동읍로 670번길 6 
...                             ...
7529          경상남도 합천군 용주면 성산3길 31 
7530           경상남도 합천군 용주면 성산길 18 
7531          경상남도 합천군 용주면 손목2길 13 
7532       경상남도 합천군 용주면 황계폭포로 1395 
7533          경상남도 합천군 용주면 손목3길 74 

[7534 rows x 1 columns]>

In [46]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in senior_center.iterrows():
    address = row['전체 주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"주소": address, "경도": x, "위도": y})
        else:
            coordinates.append({"주소": address, "경도": None, "위도": None})
    else:
        coordinates.append({"주소": address, "경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 출력
print(coord_df)

                                 주소             경도            위도
0     경상남도 창원시 의창구 동읍 신촌본포로442번길 26  128.644385896  35.359032004
1          경상남도 창원시 의창구 동읍 백월로 540   128.654864588  35.305038793
2     경상남도 창원시 의창구 동읍 동읍로 359번길 43   128.661179593  35.298651114
3      경상남도 창원시 의창구 동읍 동읍로 835번길 7   128.666214508  35.333315727
4      경상남도 창원시 의창구 동읍 동읍로 670번길 6   128.664352833  35.323306805
...                             ...            ...           ...
7529          경상남도 합천군 용주면 성산3길 31   128.135081348  35.556141195
7530           경상남도 합천군 용주면 성산길 18   128.148858647  35.552522829
7531          경상남도 합천군 용주면 손목2길 13   128.127954252  35.553945293
7532       경상남도 합천군 용주면 황계폭포로 1395   128.130041340  35.552212500
7533          경상남도 합천군 용주면 손목3길 74   128.126910687  35.555629547

[7534 rows x 3 columns]
